In [1]:
import pandas as pd
import numpy as np

# Example dataset for classification
# Features: height (cm), weight (kg), shoe size (US)
# Labels: gender (male/female)

data = {'height': [181, 177, 160, 154, 166, 190, 175, 177, 159, 171, 181],
        'weight': [80, 70, 60, 54, 65, 90, 64, 70, 55, 75, 85],
        'shoe_size': [11, 10, 7, 6, 9, 12, 9, 10, 7, 9, 11],
        'gender': ['male', 'male', 'female', 'female', 'male', 'male', 'female', 'female', 'female', 'male', 'male']}

df = pd.DataFrame(data)

print(df)

    height  weight  shoe_size  gender
0      181      80         11    male
1      177      70         10    male
2      160      60          7  female
3      154      54          6  female
4      166      65          9    male
5      190      90         12    male
6      175      64          9  female
7      177      70         10  female
8      159      55          7  female
9      171      75          9    male
10     181      85         11    male


Encode

In [2]:
gender_map ={'male':0,'female':1}
df['gender'] = df['gender'].map(gender_map)

In [3]:
df

,height,weight,shoe_size,gender
0,181,80,11,0
1,177,70,10,0
2,160,60,7,1
3,154,54,6,1
4,166,65,9,0
5,190,90,12,0
6,175,64,9,1
7,177,70,10,1
8,159,55,7,1
9,171,75,9,0


Start model

In [4]:
df = np.array(df)
X = df[:,:3]
y = df[:,-1]


In [5]:
X =X.T
X

array([[181, 177, 160, 154, 166, 190, 175, 177, 159, 171, 181],
       [ 80,  70,  60,  54,  65,  90,  64,  70,  55,  75,  85],
       [ 11,  10,   7,   6,   9,  12,   9,  10,   7,   9,  11]],
      dtype=int64)

In [6]:
y

array([0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0], dtype=int64)

In [16]:
num_features = X.shape[0] #3
n_sample = X.shape[1]

layer1 = 10
layer2 = 10
layer3 = 1



def intital():
    W1 = np.random.randn(layer1,num_features)
    b1 = np.random.randn(layer1,1)-0.5

    W2 = np.random.randn(layer2,layer1)
    b2 = np.random.randn(layer2,1)-0.5

    W3 = np.random.randn(layer3,layer2) #output layer
    b3 = np.random.randn(layer3,1)-0.5

    return W1, b1, W2, b2, W3,b3

# forwards 
def Relu(Z):
    return np.maximum(0, Z)

def Sigmoid(Z):
    return 1 / (1 + np.exp(-Z))


def forwards(X,W1,W2,W3,b1,b2,b3):
    A0 = X 

    Z1 = W1.dot(A0) + b1
    A1 = Relu(Z1)

    Z2 = W2.dot(A1) + b2
    A2 = Relu(Z2)

    Z3 = W3.dot(A2) + b3
    A3 = Sigmoid(Z3)

    return Z1,A1,Z2,A2,Z3,A3



# backwards 
def Relu_deriv(Z):
    return Z>0 

def adjust_output(A3):
    A3 = (A3 >= 0.5).astype(int)
    return A3


def backwards(Z1, A1, Z2, A2, Z3, A3, W3, W2, X, y):
    A3 = adjust_output(A3)
    
    dZ3 = 2*(A3 - y)  /n_sample  
    dW3 = dZ3.dot(A2.T) 
    db3 = np.sum(dZ3) 

    dZ2 = W3.T.dot(dZ3) * Relu_deriv(Z2)
    dW2 = dZ2.dot(A2.T) 
    db2 = np.sum(dZ2) 

    dZ1 = W2.T.dot(dZ2) *Relu_deriv(Z1)
    dW1 = dZ1.dot(X.T) 
    db1 = np.sum(dZ2) 
    
    return dW3, db3, dW2, db2, dW1, db1

def update_para(W1, b1, W2, b2,W3,b3,dW1,db1,dW2,db2,dW3,db3,learning_rate=0.01):
    W3 = W3 - learning_rate*dW3
    b3 = b3 - learning_rate*db3
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    return W1, b1, W2, b2, W3,b3


#model 
def get_accuracy(y_val, y_test):
    correct_predictions = np.sum(y_val == y_test)
    total_predictions = len(y_test)
    print(correct_predictions / total_predictions)


def model(X,y):
    W1, b1, W2, b2, W3,b3 = intital()
    for i in range(1000):
        Z1,A1,Z2,A2,Z3,A3 = forwards(X,W1,W2,W3,b1,b2,b3)
        dW3, db3, dW2, db2, dW1, db1 = backwards(Z1, A1, Z2, A2, Z3, A3, W3, W2, X, y)
        W1, b1, W2, b2, W3,b3 = update_para(W1, b1, W2, b2,W3,b3,dW1,db1,dW2,db2,dW3,db3)

    return W1, b1, W2, b2, W3, b3, A1, A2, A3

def predict(X,W1,W2,W3,b1,b2,b3):
    Z1,A1,Z2,A2,Z3,A3 = forwards(X,W1,W2,W3,b1,b2,b3)
    return A3
    



In [17]:
W1, b1, W2, b2, W3, b3, A1, A2, A3 = model(X,y)

In [18]:
df_test = pd.DataFrame({'height': [169, 185, 148, 195, 160, 170, 172, 187, 198, 163],
                        'weight': [58, 90, 45, 104, 56, 75, 67, 80, 110, 59],
                        'shoe_size': [8, 11, 5, 13, 6, 9, 9, 10, 14, 7],
                        'gender': ['female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'male', 'female']})

df_test['gender'] = df_test['gender'].map(gender_map)

df_test = np.array(df_test)
X_test = df_test[:,:3]
y_test = df_test[:,-1]

X_test = X_test.T


In [19]:
y_pred = predict(X_test,W1,W2,W3,b1,b2,b3)

In [20]:
get_accuracy(adjust_output(y_pred),y_test)

0.5


In [21]:
adjust_output(y_pred)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
y_test

array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int64)